# In this notebook we will find cell-cell interaction changed genes

---
First we need to import our spatial pre-processing notebook and run binSpect below:

In [ ]:
source("scripts//spatial_pre_process.R")

# identify genes with a spatial coherent expression profile
km_spatialgenes = binSpect(my_giotto_object, bin_method = 'kmeans')

### 1. Interaction changed genes

Below, we'll find our [interaction changed genes](https://rubd.github.io/Giotto_site/reference/findICG.html) (ICGs):

In [ ]:
## select top 25th highest expressing genes
gene_metadata = fDataDT(my_giotto_object)
plot(gene_metadata$nr_cells, gene_metadata$mean_expr)
plot(gene_metadata$nr_cells, gene_metadata$mean_expr_det)

quantile(gene_metadata$mean_expr_det)
high_expressed_genes = gene_metadata[mean_expr_det > 4]$gene_ID

## identify genes that are associated with proximity to other cell types
ICGscoresHighGenes =  findICG(gobject = my_giotto_object,
                              selected_genes = high_expressed_genes,
                              spatial_network_name = 'Delaunay_network',
                              cluster_column = 'cell_types',
                              diff_test = 'permutation',
                              adjust_method = 'fdr',
                              nr_permutations = 500,
                              do_parallel = T, cores = 2)

In [ ]:
## visualize all genes
plotCellProximityGenes(my_giotto_object, 
                       cpgObject = ICGscoresHighGenes, 
                       method = 'dotplot')

### 2. Filter ICGs:

Below we'll [filter our ICGs](https://rubd.github.io/Giotto_site/reference/filterICG.html):

In [ ]:
# filter genes
ICGscoresFilt = filterICG(ICGscoresHighGenes, 
                          min_cells = 2, min_int_cells = 2, min_fdr = 0.1,
                          min_spat_diff = 0.1, min_log2_fc = 0.1, min_zscore = 1)

### 3. Visualize
Now we can visualize our selected ICGs:


In [ ]:
## visualize subset of interaction changed genes (ICGs)
# random subset
ICG_genes = c("Atp1b2", "Slc17a7", "Pcdh8")
ICG_genes_types = c("cell_type_2", "cell_type_3", "cell_type_4")
names(ICG_genes) = ICG_genes_types

plotICG(gobject = my_giotto_object,
        cpgObject = ICGscoresHighGenes,
        source_type = "cell_type_1",
        source_markers = c("Cldn5", "Lingo1"),
        ICG_genes = ICG_genes)